# Processing the Data

In [1]:
import torch

from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"

# First time loading
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained("tokenizers/" + checkpoint, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained("models/" + checkpoint, local_files_only=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
tokenizer.save_pretrained("tokenizers/" + checkpoint)
model.save_pretrained("models/" + checkpoint)

In [4]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

In [5]:
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()


## Downloading and Caching the Data

In [6]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc") # Dataset name
raw_datasets

Downloading: 28.8kB [00:00, 14.3MB/s]                   
Downloading: 28.7kB [00:00, 14.4MB/s]                   


Downloading: 6.22kB [00:00, 6.21MB/s]
Downloading: 1.05MB [00:00, 1.05MB/s]
Downloading: 441kB [00:00, 533kB/s]
100%|██████████| 3/3 [00:07<00:00,  2.63s/it]


Dataset glue downloaded and prepared to C:\Users\liana\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 52.63it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

Access each pair of sentences in our `raw_datasets` object by indexing like a dictionary.

In [7]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0,
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

Find out which integer corresponds to which label.

In [8]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

In [9]:
raw_train_dataset[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'label': 0,
 'idx': 16,
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .'}

In [10]:
raw_train_dataset[87]

{'sentence1': 'Tuition at four-year private colleges averaged $ 19,710 this year , up 6 percent from 2002 .',
 'label': 1,
 'idx': 100,
 'sentence2': 'For the current academic year , tuition at public colleges averaged $ 4,694 , up almost $ 600 from the year before .'}

## Preprocessing Dataset

Tokenise data: can feed the tokeniser with one sentence or a list of sentences.

In [13]:
from transformers import AutoTokenizer

tokenized_sentences_1 = tokenizer(raw_train_dataset["sentence1"])
tokenized_sentences_2 = tokenizer(raw_train_dataset["sentence2"])

But cannot pass two sequences for predicting whether they are paraphrases. These two sequences must be handled as a pair with appropriate processing.

In [14]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Separate tokenising

In [16]:
tokenizer(raw_train_dataset[15]["sentence1"])

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tokenizer(raw_train_dataset[15]["sentence2"])

{'input_ids': [101, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Pair tokenising

In [18]:
tokenizer(raw_train_dataset[15]["sentence1"], raw_train_dataset[15]["sentence2"])

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

BERT is pretrained with token type IDs, and on top of the masked language modeling objective we talked about in Chapter 1, it has an additional objective called next sentence prediction. The goal with this task is to model the relationship between pairs of sentences.

In [23]:
# First sentence is 0, second sentence 1, for BERT
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

In [24]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

# Disadvantage
# Returns a dicionary
# Only work if enough RAM available to store the whole dataset

### Multi-processing

Using the `map()` function. This function applies a function on each element on a dataset.

In [25]:
# Takes a dictionary and returns a new dictionary with the keys input_ids, attention_mask, token_type_ids
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Padding all samples to the maximum length is not efficient. Pad the samples when building a batch (max length of that batch). The preprocessing function can be used to add new fields to all datasets or change data values for existing keys.

In [26]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # The function is applied to multiple elements in the dataset at once (faster)
tokenized_datasets

100%|██████████| 2/2 [00:00<00:00, 13.61ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

### Dynamic Padding

The function that puts samples together inside a batch is called a **collate function**, an argument that can be passed to a `DataLoader`. The default function will convert samples to PyTorch tensors and concatenate them. That's why padding must be postponed because of the different sizes. (Can cause problems when training on TPUs as they prefer fixed shapes. Use extra padding.) We need to define a collate function that applies an appropriate amount of padding.

In [27]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Test some samples. Remove `idx`, `sentence1` and `sentence2` because they contain strings and strings cannot be converted into tensors. We already have their tokens.

In [28]:
samples = tokenized_datasets["train"][:8] # One batch
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]} # Create a dictionary from dict comprehension
[len(x) for x in samples["input_ids"]] # Length of each entry in the batch

[50, 59, 47, 67, 59, 50, 62, 32]

In [30]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'labels': torch.Size([8])}

# Fine-tuning With the Trainer API

## Training

Define training arguments (hyperparameters). Only mandatory argument is a directory to save the model and checkpoints. The rest has defaults for basic fine-tuning.

In [31]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

## Define Our Model

In [32]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator, # Default data_collator is DataCollatorWithPadding so no need define also can
    tokenizer=tokenizer,
    # evaluation_strategy="steps" or "epoch"
    # compute_metrics()
)

In [34]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377
 36%|███▋      | 500/1377 [01:33<02:39,  5.48it/s]Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json


{'loss': 0.53, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
 73%|███████▎  | 1000/1377 [03:09<01:07,  5.58it/s]Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json


{'loss': 0.2913, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


Model weights saved in test-trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1000\special_tokens_map.json
100%|██████████| 1377/1377 [04:20<00:00,  5.81it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1377/1377 [04:20<00:00,  5.29it/s]

{'train_runtime': 260.2468, 'train_samples_per_second': 42.283, 'train_steps_per_second': 5.291, 'train_loss': 0.3381862238687974, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3381862238687974, metrics={'train_runtime': 260.2468, 'train_samples_per_second': 42.283, 'train_steps_per_second': 5.291, 'train_loss': 0.3381862238687974, 'epoch': 3.0})

## Evaluation

Building a `compute_metrics()`. The output of the predict() method is another named tuple with three fields: predictions, label_ids, and metrics. The metrics will contain loss on the dataset passed.

In [35]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8
 98%|█████████▊| 50/51 [00:01<00:00, 25.58it/s]

(408, 2) (408,)


100%|██████████| 51/51 [00:21<00:00, 25.58it/s]

All Transformer models return logits. Transform logits into prediction by taking the index with the maximum value on the second axis.

In [36]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

Compare `preds` to the labels. The table in the BERT paper reported an F1 score of 88.9 for the base model. That was the uncased model while we are currently using the cased model, which explains the better result.

In [37]:
from datasets import load_metric

metric = load_metric("glue", "mrpc") # Benchmark name and dataset name
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8480392156862745, 'f1': 0.8952702702702703}

Combining everything togther.

In [38]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [39]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # Our function
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\liana/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads"

Note that we create a new TrainingArguments with its evaluation_strategy set to "epoch" and a new model — otherwise, we would just be continuing the training of the model we have already trained. The exact accuracy/F1 score you reach might be a bit different from what we found, because of the random head initialization of the model, but it should be in the same ballpark.

In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377
 33%|███▎      | 459/1377 [02:25<05:34,  2.74it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8

 33%|███▎      | 459/1377 [02:30<05:34,  2.74it/s]

{'eval_loss': 0.48459067940711975, 'eval_accuracy': 0.8137254901960784, 'eval_f1': 0.8694158075601375, 'eval_runtime': 4.5448, 'eval_samples_per_second': 89.772, 'eval_steps_per_second': 11.222, 'epoch': 1.0}


 36%|███▋      | 500/1377 [02:40<03:16,  4.47it/s]Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json


{'loss': 0.5664, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
 67%|██████▋   | 918/1377 [04:48<01:34,  4.87it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8

 67%|██████▋   | 919/1377 [04:52<09:10,  1.20s/it]

...

 73%|███████▎  | 1000/1377 [05:06<01:05,  5.76it/s]Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json


{'loss': 0.3728, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


Model weights saved in test-trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1000\special_tokens_map.json
100%|██████████| 1377/1377 [06:17<00:00,  5.56it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8

100%|██████████| 1377/1377 [06:21<00:00,  5.56it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1377/1377 [06:21<00:00,  3.61it/s]

{'eval_loss': 0.7744005918502808, 'eval_accuracy': 0.8235294117647058, 'eval_f1': 0.8762886597938144, 'eval_runtime': 3.3578, 'eval_samples_per_second': 121.507, 'eval_steps_per_second': 15.188, 'epoch': 3.0}
{'train_runtime': 381.3579, 'train_samples_per_second': 28.855, 'train_steps_per_second': 3.611, 'train_loss': 0.40473202751785753, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.40473202751785753, metrics={'train_runtime': 381.3579, 'train_samples_per_second': 28.855, 'train_steps_per_second': 3.611, 'train_loss': 0.40473202751785753, 'epoch': 3.0})

The Trainer will work out of the box on multiple GPUs or TPUs and provides lots of options, like mixed-precision.

# A Full Fine-Tuning Without the Trainer API